In [ ]:
from google.colab import drive
import os
import pandas as pd
import joblib
from datetime import datetime


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PASTA_BASE = "/content/drive"

In [ ]:
PASTA_BASE = "/content/drive"
ARQUIVO_TREINO = os.path.join(PASTA_BASE, "arquivo.csv")
COLUNA_TEXTO = "coluna_x"
COLUNA_LABEL = "coluna_y" # 1 = X, 0 = não X

# Dados para classificação em lote
EXTENSOES_VALIDAS = (".csv", ".xlsx")


# Limiares para revisão humana
LIMIAR_CONFIANTE = 0.90  # Ajustado de 0.80
LIMIAR_DUVIDOSO = 0.40   # Ajustado de 0.50


# Nomes dos artefatos
ARQ_VECTORIZER = os.path.join(PASTA_BASE, "vectorizer.joblib")
ARQ_MODEL = os.path.join(PASTA_BASE, "model.joblib")

print("Todas as variáveis de configuração foram definidas em uma única célula.")

Todas as variáveis de configuração foram definidas em uma única célula.


In [ ]:
def treinar_e_salvar_modelo():
    training_dfs = []

    # Iterar sobre todos os arquivos na PASTA_BASE
    for f in os.listdir(PASTA_BASE):
        caminho_arquivo = os.path.join(PASTA_BASE, f)
        nome_base, extensao = os.path.splitext(f)

        # Verificar se é um arquivo válido para treinamento
        # Removida a exclusão explícita de ARQUIVO_TREINO para que ele seja incluído no loop
        if (os.path.isfile(caminho_arquivo) and
            extensao.lower() in EXTENSOES_VALIDAS and
            "_classificado" not in nome_base):

            print(f"Tentando carregar arquivo para treino: {caminho_arquivo}")
            temp_df = None
            try:
                if extensao.lower() == ".csv":
                    try:
                        temp_df = pd.read_csv(caminho_arquivo, sep=';')
                    except Exception:
                        temp_df = pd.read_csv(caminho_arquivo, sep=',')
                elif extensao.lower() == ".xlsx":
                    temp_df = pd.read_excel(caminho_arquivo)

                if temp_df is not None:
                    # Verificar se as colunas necessárias existem
                    if COLUNA_TEXTO not in temp_df.columns or COLUNA_LABEL not in temp_df.columns:
                        print(f"⚠️ Aviso: Arquivo '{f}' não contém as colunas '{COLUNA_TEXTO}' e/ou '{COLUNA_LABEL}'. Pulando este arquivo.")
                        continue

                    training_dfs.append(temp_df)

            except Exception as e:
                print(f"❌ Erro ao carregar o arquivo '{f}': {e}. Pulando este arquivo.")
                continue

    # Combinar todos os DataFrames carregados
    if not training_dfs:
        print("❌ Erro: Nenhum arquivo de treino válido encontrado ou carregado. Não é possível treinar o modelo.")
        return

    df_train = pd.concat(training_dfs, ignore_index=True)

    if df_train.empty:
        print("❌ Erro: DataFrame de treino resultante está vazio. Não é possível treinar o modelo.")
        return

    df_train[COLUNA_TEXTO] = df_train[COLUNA_TEXTO].fillna("")

    # Processar a coluna de rótulo para considerar 'x' como 1 e outros como 0
    df_train[COLUNA_LABEL] = df_train[COLUNA_LABEL].astype(str).str.strip().str.lower().apply(
        lambda val: 1 if val == "x" else 0
    )
    df_train[COLUNA_LABEL] = df_train[COLUNA_LABEL].astype(int)

    # Vectorizer
    vectorizer = TfidfVectorizer(
        lowercase=True,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )

    X_train = vectorizer.fit_transform(df_train[COLUNA_TEXTO])
    y_train = df_train[COLUNA_LABEL]

    # --- Adicionado para depuração: verificar a distribuição das classes após o processamento ---
    print("Distribuição das classes na coluna '{}' após processamento:".format(COLUNA_LABEL))
    print(y_train.value_counts())
    # --------------------------------------------------------------------------------------

    # Modelo
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Salvar artefatos
    joblib.dump(vectorizer, ARQ_VECTORIZER)
    joblib.dump(model, ARQ_MODEL)

    print("✔ Modelo treinado e salvo")
    print(ARQ_VECTORIZER)
    print(ARQ_MODEL)

In [ ]:
treinar_e_salvar_modelo()

Tentando carregar arquivo para treino: /content/drive/MyDrive/teste_marcacao/revisar/dados_treino.csv
Distribuição das classes na coluna 'excluir' após processamento:
excluir
1    711
0    371
Name: count, dtype: int64
✔ Modelo treinado e salvo
/content/drive/MyDrive/teste_marcacao/revisar/vectorizer.joblib
/content/drive/MyDrive/teste_marcacao/revisar/model.joblib


In [ ]:
def carregar_modelo():
    if not os.path.exists(ARQ_VECTORIZER) or not os.path.exists(ARQ_MODEL):
        raise FileNotFoundError("Vectorizer/Model não encontrados. Rode o BLOCO 3 primeiro.")

    vectorizer = joblib.load(ARQ_VECTORIZER)
    model = joblib.load(ARQ_MODEL)
    return vectorizer, model

In [ ]:
import re

def classificar_arquivo(caminho_arquivo, vectorizer, model):
    nome_base, extensao = os.path.splitext(os.path.basename(caminho_arquivo))

    # Carregar dados
    if extensao == ".csv":
        try:
            df = pd.read_csv(caminho_arquivo, sep=';')
        except Exception as e:
            print(f"Erro ao ler CSV com ponto e vírgula: {e}. Tentando com vírgula...")
            df = pd.read_csv(caminho_arquivo, sep=',')
    else:
        df = pd.read_excel(caminho_arquivo)

    # Verificar se a COLUNA_TEXTO existe
    if COLUNA_TEXTO not in df.columns:
        print(f"Erro: O arquivo '{caminho_arquivo}' não possui a coluna '{COLUNA_TEXTO}'. Colunas disponíveis: {df.columns.tolist()}")
        raise KeyError(f"Coluna '{COLUNA_TEXTO}' não encontrada no arquivo '{caminho_arquivo}'.")

    df[COLUNA_TEXTO] = df[COLUNA_TEXTO].fillna("")

    # Remover caracteres ilegais para XML/Excel
    illegal_char_re = re.compile(r'[\x00-\x08\x0b-\x0c\x0e-\x1f]')
    df[COLUNA_TEXTO] = df[COLUNA_TEXTO].apply(lambda x: illegal_char_re.sub('', x) if isinstance(x, str) else x)

    # Vetorização
    X = vectorizer.transform(df[COLUNA_TEXTO])

    # Predições
    probas = model.predict_proba(X)[:, 1]
    preds = model.predict(X)

    df["prob_X"] = probas
    df["marcacao_X"] = pd.Series(preds).map({1: "x", 0: ""})

    # Revisão humana - Lógica invertida para CONFIANTE e DESCARTADO
    df["status_revisao"] = df["prob_X"].apply(
        lambda p:
            (
                "DESCARTADO" if p >= LIMIAR_CONFIANTE else  # Se alta probabilidade de ser 'X' (excluir), marca como DESCARTADO
                "REVISAR" if p >= LIMIAR_DUVIDOSO else      # Se probabilidade intermediária, marca como REVISAR
                "CONFIANTE"                                 # Se baixa probabilidade de ser 'X' (não excluir), marca como CONFIANTE
            )
    )

    # Versionamento
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Cria a pasta 'resultados' se não existir
    pasta_resultados = os.path.join(PASTA_BASE, "resultados")
    os.makedirs(pasta_resultados, exist_ok=True)

    saida = os.path.join(
        pasta_resultados,
        f"{nome_base}_classificado_{timestamp}.xlsx"
    )

    df.to_excel(saida, index=False)
    return saida

In [ ]:
def classificar_pasta():
    vectorizer, model = carregar_modelo()

    arquivos = [
        f for f in os.listdir(PASTA_BASE)
        if f.endswith(EXTENSOES_VALIDAS)
        and not f.endswith("_classificado.xlsx")
        and os.path.join(PASTA_BASE, f) != ARQUIVO_TREINO  # Excluir o arquivo de treino
    ]

    if not arquivos:
        print("Nenhum arquivo para classificar.")
        return

    saidas = []
    for f in arquivos:
        caminho = os.path.join(PASTA_BASE, f)
        print(f"Tentando classificar arquivo: {caminho}") # Adicionado para depuração
        try:
            saida = classificar_arquivo(caminho, vectorizer, model)
            saidas.append(saida)
        except (KeyError, pd.errors.ParserError) as e:
            print(f"❌ Erro ao classificar o arquivo {caminho}: {e}. Pulando este arquivo.")
        except Exception as e:
            print(f"❌ Ocorreu um erro inesperado ao classificar o arquivo {caminho}: {e}. Pulando este arquivo.")

    print("✔ Classificação em lote concluída")
    for s in saidas:
        print("Gerado:", s)

In [ ]:
import os
print(f"Conteúdo da pasta '{PASTA_BASE}':")
conteudo_pasta = os.listdir(PASTA_BASE)
if not conteudo_pasta:
    print("  (Vazio)")
else:
    for item in conteudo_pasta:
        print(f"  - {item}")

Conteúdo da pasta '/content/drive/MyDrive/teste_marcacao/revisar/':
  - dados_treino.csv
  - conteudo_lote_17_unificado_classificado_20260113_232900.xlsx
  - conteudo_lote_17_unificado (1)_classificado_20260113_232904.xlsx
  - conteudo_lote_18_unificado_classificado_20260113_232907.xlsx
  - conteudo_lote_21_unificado_classificado_20260113_232910.xlsx
  - conteudo_lote_20_unificado_classificado_20260113_232912.xlsx
  - conteudo_lote_19_unificado_classificado_20260113_232914.xlsx
  - conteudo_lote_15_unificado_classificado_20260113_232917.xlsx
  - conteudo_lote_16_unificado_classificado_20260113_232921.xlsx
  - conteudo_lote_0406_unificado_classificado_20260113_232928.xlsx
  - conteudo_lote_24_unificado_classificado_20260113_232933.xlsx
  - conteudo_lote_25_unificado_classificado_20260113_232936.xlsx
  - conteudo_lote_0710_unificado_classificado_20260113_232949.xlsx
  - conteudo_lote_1114_unificado_classificado_20260113_233006.xlsx
  - conteudo_lote_22_unificado_classificado_20260113_233

In [ ]:
classificar_pasta()

Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_17_unificado_classificado_20260113_232900.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_17_unificado (1)_classificado_20260113_232904.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_18_unificado_classificado_20260113_232907.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_21_unificado_classificado_20260113_232910.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_20_unificado_classificado_20260113_232912.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_19_unificado_classificado_20260113_232914.xlsx
Tentando classificar arquivo: /content/drive/MyDrive/teste_marcacao/revisar/conteudo_lote_15_unificado_classificado_20260113_232917.xlsx
Tentando classificar arquivo: /conten